

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_SUPPLEMENT_CLINICAL.ipynb)




# **Extract neurologic deficits related to Helath Stroke Scale (NIHSS)**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



## 1. Colab Setup

**Import license keys**

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

## 2. Install dependencies

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

**Import dependencies into Python and start the Spark session**

In [4]:
# Import sparknlp & sparknlp_jsl packages
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

# Import Pyspark packages
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.ml import Pipeline, PipelineModel

import numpy as np
import pandas as pd

spark = sparknlp_jsl.start(license_keys['SECRET'])

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 3.4.3
Spark NLP_JSL Version : 3.5.1


## 3. Select the NER model and construct the pipeline

In [5]:
MODEL_NAME = "ner_nihss"

**Create the pipeline**

In [6]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models') \
     .setInputCols(['sentence', 'token']) \
     .setOutputCol('embeddings')

ner = MedicalNerModel.pretrained(MODEL_NAME, 'en', 'clinical/models') \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\


ner_pipeline = Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        embeddings,
        ner,
        ner_converter
        ])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = ner_pipeline.fit(empty_df)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_nihss download started this may take some time.
Approximate size to download 13.8 MB
[OK!]


## 4. Create example inputs

In [7]:
sample_text = ["""Abdomen , soft , nontender . NIH stroke scale on presentation was 23 to 24 for , one for consciousness , two for month and year and two for eye / grip , one to two for gaze , two for face , eight for motor , one for limited ataxia , one to two for sensory , three for best language and two for attention . On the neurologic examination the patient was intermittently ."""]

## 5. Use the pipeline to create outputs

In [8]:
df = spark.createDataFrame(pd.DataFrame({'text': sample_text}))

result = pipeline_model.transform(df)

result.select(F.explode(F.arrays_zip(result.ner_chunk.result, result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(30, truncate=False)

+----------------+---------------+
|chunk           |ner_label      |
+----------------+---------------+
|NIH stroke scale|NIHSS          |
|23 to 24        |Measurement    |
|one             |Measurement    |
|consciousness   |1a_LOC         |
|two             |Measurement    |
|month and year  |1b_LOCQuestions|
|two             |Measurement    |
|eye / grip      |1c_LOCCommands |
|one             |Measurement    |
|two             |Measurement    |
|gaze            |2_BestGaze     |
|two             |Measurement    |
|face            |4_FacialPalsy  |
|eight           |Measurement    |
|one             |Measurement    |
|limited ataxia  |7_LimbAtaxia   |
|one to two      |Measurement    |
|sensory         |8_Sensory      |
|three           |Measurement    |
|best language   |9_BestLanguage |
|two             |Measurement    |
+----------------+---------------+



## 6. Visualize results

In [9]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)